#START: LEXI CHEN

# Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

## Load Dataset


Test set: the last month of data provided: December

Training set: a minimum of 6 months of data prior to the test month
You may use more months for training if you believe it improves model stability and performance

In [2]:
test_raw = pd.read_csv("../raw_data/CRMLSSold202512.csv")

df_06 = pd.read_csv("../raw_data/CRMLSSold202506.csv")
df_07 = pd.read_csv("../raw_data/CRMLSSold202507.csv")
df_08 = pd.read_csv("../raw_data/CRMLSSold202508.csv")
df_09 = pd.read_csv("../raw_data/CRMLSSold202509.csv")
df_10 = pd.read_csv("../raw_data/CRMLSSold202510.csv")
df_11 = pd.read_csv("../raw_data/CRMLSSold202511.csv")

train_raw = pd.concat([df_06, df_07, df_08, df_09, df_10, df_11],ignore_index=True
)

print("Raw train shape:", train_raw.shape)
print("Raw test shape:", test_raw.shape)

/var/folders/_p/mv7dq2pn2_57jhgzwjp8dlrw0000gn/T/ipykernel_79785/1293242065.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_06 = pd.read_csv("../raw_data/CRMLSSold202506.csv")


Raw train shape: (134265, 78)
Raw test shape: (20538, 78)


In [3]:
train_df = train_raw.copy()
test_df = test_raw.copy()
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (134265, 78)
Test shape: (20538, 78)


## Data Scope



The following records are removed from both training and test sets:

- `ClosePrice ≤ 0`
- `LivingArea ≤ 0`
- Missing `Latitude` or `Longitude`

These records do not represent valid market transactions and are treated as data-quality errors rather than statistical outliers.

---



To reduce the impact of extreme or potentially erroneous sales prices,  
we remove:

- The bottom 0.5% of `ClosePrice`
- The top 0.5% of `ClosePrice`
- The percentile thresholds are computed separately for the training and test sets.

---
The columns `latfilled` and `lonfilled` are metadata flags indicating whether latitude and longitude were originally missing and later filled using the Google Maps API.

These fields:

- Do not describe property characteristics
- Do not represent economic or structural information
- Are only used for auditing data quality

Including these variables in modeling could introduce unintended bias by allowing the model to learn patterns related to data collection issues rather than true market behavior.

Therefore, they are removed prior to analysis and modeling.


---
Because the model must support both on-market and off-market predictions, 
we exclude features that are only available when a property is actively listed.

The following fields are removed:

- `ListPrice`
- `OriginalListPrice`

These variables are only present during an active listing and are highly correlated with final sale price. Including them would:

- Introduce data leakage
- Inflate model performance
- Prevent use of the model for off-market properties

Additionally, any other listing-only or marketing-related features are excluded to ensure the model relies only on stable, property-level characteristics.


In [4]:
def base_clean(df):
    df = df[(df["PropertyType"] == "Residential") &(df["PropertySubType"] == "SingleFamilyResidence")]

    df = df[(df["ClosePrice"] > 0) &(df["LivingArea"] > 0) &(df["Latitude"].notna()) &(df["Longitude"].notna())]
    return df



train_df = base_clean(train_df)
test_df = base_clean(test_df)



# ==============================
# TRAIN-ONLY QUANTILE FIT
# ==============================

train_lower = train_df["ClosePrice"].quantile(0.005)
train_upper = train_df["ClosePrice"].quantile(0.995)

train_df = train_df[(train_df["ClosePrice"] >= train_lower) &(train_df["ClosePrice"] <= train_upper)
]

test_df = test_df[(test_df["ClosePrice"] >= train_lower) &(test_df["ClosePrice"] <= train_upper)
]



# ==============================
# COLUMN CLEANUP (CONSISTENT)
# ==============================

cols_to_drop = ["latfilled", "lonfilled"]

train_df = train_df.drop(columns=cols_to_drop, errors="ignore")
test_df = test_df.drop(columns=cols_to_drop, errors="ignore")

# drop anything with listprice in name
listprice_cols = [c for c in train_df.columns if "listprice" in c.lower()]

train_df = train_df.drop(columns=listprice_cols, errors="ignore")
test_df = test_df.drop(columns=listprice_cols, errors="ignore")



# ==============================
# FINAL SHAPES (STABLE EVERY RUN)
# ==============================

print("Clean train shape:", train_df.shape)
print("Clean test shape:", test_df.shape)

Clean train shape: (67746, 76)
Clean test shape: (10331, 76)


In [5]:
test_df

,BuyerAgentAOR,ListAgentAOR,Flooring,ViewYN,WaterfrontYN,BasementYN,PoolPrivateYN,ListingKey,ListAgentEmail,CloseDate,...,LotSizeDimensions,LotSizeArea,MainLevelBedrooms,NewConstructionYN,GarageSpaces,HighSchoolDistrict,PostalCode,AssociationFee,LotSizeSquareFeet,MiddleOrJuniorSchoolDistrict
0,ContraCosta,ContraCosta,"Carpet,Tile,Wood",NaN,NaN,NaN,False,1150041639,teresa@teresahooper.com,2025-12-31,...,NaN,10080.0,NaN,False,3.0,San Ramon Valley,94596,975.0,10080.0,NaN
2,SanDiego,SanDiego,"Carpet,Wood",True,NaN,NaN,False,1150038683,laura@lauralothianrealestate.com,2025-12-31,...,NaN,34745.0,NaN,False,3.0,NaN,91364,NaN,34745.0,NaN
3,Mlslistings,Mlslistings,NaN,False,NaN,NaN,NaN,1150038607,trung.lam@kw.com,2025-12-31,...,NaN,6600.0,NaN,False,2.0,Other,95121,NaN,6600.0,NaN
7,Mlslistings,Mlslistings,NaN,False,NaN,NaN,NaN,1150032869,vickie@realsmartgroup.com,2025-12-31,...,NaN,8262.0,NaN,False,1.0,San Jose Unified,95124,NaN,8262.0,NaN
9,Mlslistings,Mlslistings,NaN,False,NaN,NaN,NaN,1150028403,vickie@realsmartgroup.com,2025-12-31,...,NaN,9222.0,NaN,False,2.0,Other,95128,NaN,9222.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20490,Mlslistings,Mlslistings,"Carpet,Stone,Wood",True,NaN,NaN,NaN,1052311956,rick.smith@camoves.com,2025-12-01,...,NaN,1745449.0,NaN,False,0.0,Carmel Unified,93924,NaN,1745449.0,NaN
20494,Glendale,Glendale,"Tile,Wood",True,NaN,NaN,False,1048440089,narbe.avanessian@evrealestate.com,2025-12-05,...,NaN,105281.0,3.0,False,3.0,Antelope Valley Union,93551,0.0,105281.0,NaN
20510,NorthSanLuisObispo,NorthSanLuisObispo,"Carpet,Laminate",True,NaN,NaN,False,1035678879,carolynne@countryrealestate.com,2025-12-17,...,20AC,20.0,3.0,False,0.0,King City Joint Union,93426,0.0,871200.0,NaN
20511,TheInlandGateway,TheInlandGateway,NaN,True,NaN,NaN,False,1035520905,brian@cohen-realty.com,2025-12-15,...,NaN,9620.0,3.0,False,1.0,Rim of the World,92352,0.0,9620.0,NaN


## Missing Value Handling Strategy

To ensure model stability and reliability, we handle missing values using a two-step approach:

1. Drop High-Missing Columns

Columns with more than 30% missing values are removed because when a large portion of data is missing, the feature becomes unreliable.

We use a 30% threshold as a practical balance:
- Below 30% → information is still mostly intact.
- Above 30% → risk of distortion increases significantly.

This threshold is commonly used in applied machine learning when no domain-specific rule exists.

---

2. Random Imputation for Low-Missing Columns

For columns with ≤ 30% missing values:

- Missing values are replaced by randomly sampling existing values from the same column.
- This is done separately for training and test sets (no data leakage).

Why random sampling instead of mean/median?

- It preserves the original distribution.
- It avoids shrinking variance (which mean imputation causes).
- It works well for skewed real estate variables.
- It prevents artificially concentrating values around the mean.

This method maintains the statistical structure of the data.

In [6]:
np.random.seed(42)  # For reproducibility

threshold = 0.30  # Drop columns if more than 30% missing


# ==============================
# TRAIN DATA
# ==============================

# Calculate missing percentage per column
missing_pct_train = train_df.isna().mean()

# Identify columns with high missing values (>30%)
cols_to_drop_train = missing_pct_train[missing_pct_train > threshold].index

# Drop high-missing columns
train_df = train_df.drop(columns=cols_to_drop_train)

# Identify remaining columns (≤30% missing)
cols_to_fill_train = missing_pct_train[missing_pct_train <= threshold].index

# Randomly impute missing values
for col in cols_to_fill_train:
    if train_df[col].isna().sum() > 0:
        train_df[col] = train_df[col].apply(
            lambda x: np.random.choice(train_df[col].dropna()) if pd.isna(x) else x
        )


# ==============================
# TEST DATA (separate handling)
# ==============================

missing_pct_test = test_df.isna().mean()

cols_to_drop_test = missing_pct_test[missing_pct_test > threshold].index

test_df = test_df.drop(columns=cols_to_drop_test)

cols_to_fill_test = missing_pct_test[missing_pct_test <= threshold].index

for col in cols_to_fill_test:
    if test_df[col].isna().sum() > 0:
        test_df[col] = test_df[col].apply(
            lambda x: np.random.choice(test_df[col].dropna()) if pd.isna(x) else x
        )


print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (67746, 51)
Test shape: (10331, 51)


## Feature Selection Strategy

1. Manual (domain-driven) filtering  
2. Statistical (data-driven) filtering  



In [7]:
print(train_df.columns)

Index(['BuyerAgentAOR', 'ListAgentAOR', 'ViewYN', 'PoolPrivateYN',
       'ListingKey', 'ListAgentEmail', 'CloseDate', 'ClosePrice',
       'ListAgentFirstName', 'ListAgentLastName', 'Latitude', 'Longitude',
       'UnparsedAddress', 'PropertyType', 'LivingArea', 'DaysOnMarket',
       'ListOfficeName', 'BuyerOfficeName', 'ListAgentFullName',
       'BuyerAgentMlsId', 'BuyerAgentFirstName', 'BuyerAgentLastName',
       'ListingKeyNumeric', 'MLSAreaMajor', 'CountyOrParish', 'MlsStatus',
       'AttachedGarageYN', 'ParkingTotal', 'PropertySubType', 'LotSizeAcres',
       'BuyerOfficeAOR', 'YearBuilt', 'StreetNumberNumeric', 'ListingId',
       'BathroomsTotalInteger', 'City', 'BedroomsTotal',
       'ContractStatusChangeDate', 'PurchaseContractDate',
       'ListingContractDate', 'StateOrProvince', 'FireplaceYN', 'Stories',
       'Levels', 'LotSizeArea', 'NewConstructionYN', 'GarageSpaces',
       'HighSchoolDistrict', 'PostalCode', 'AssociationFee',
       'LotSizeSquareFeet'],
      d

### Manual Feature Dropping 

Before applying statistical feature selection, we first remove features based on
business logic and modeling constraints.

Because the model must support **both on-market and off-market predictions**,
we remove any feature that:

- Is only available during listing or transaction
- Contains agent or brokerage information
- Is an identifier
- Causes data leakage
- Allows memorization instead of generalization

The goal is to keep only **property-intrinsic features**.

---

1. Remove Listing / Transaction Process Features

These exist only when a property is actively listed.

- `DaysOnMarket`
- `ListingContractDate`
- `PurchaseContractDate`
- `ContractStatusChangeDate`
- `MlsStatus`


Unavailable for off-market predictions and may introduce leakage.

---

2. Remove Agent / Office Information

These describe who handled the transaction, not the property.

- `BuyerAgentAOR`
- `ListAgentAOR`
- `ListAgentEmail`
- `ListAgentFirstName`
- `ListAgentLastName`
- `ListAgentFullName`
- `BuyerAgentMlsId`
- `BuyerAgentFirstName`
- `BuyerAgentLastName`
- `ListOfficeName`
- `BuyerOfficeName`
- `BuyerOfficeAOR`


Not available off-market and may cause overfitting.

---

3. Remove Identifiers

- `ListingKey`
- `ListingKeyNumeric`
- `ListingId`
- `StreetNumberNumeric`


Identifiers allow memorization and do not generalize.

---

4. Remove Raw Address

- `UnparsedAddress`


High cardinality and risk of memorization.  
We retain structured geographic variables instead (e.g., Latitude, Longitude, City, PostalCode).

---


After manual filtering, the remaining features:

- Describe intrinsic property characteristics
- Are available off-market
- Do not leak transaction information
- Are safe for generalizable modeling

In [8]:
# -------------------------------
# Manual Domain-Driven Dropping
# Apply to BOTH train and test
# -------------------------------

manual_drop_cols = [
    # Listing / transaction process
    "DaysOnMarket",
    "ListingContractDate",
    "PurchaseContractDate",
    "ContractStatusChangeDate",
    "MlsStatus",

    # Agent / office information
    "BuyerAgentAOR",
    "ListAgentAOR",
    "ListAgentEmail",
    "ListAgentFirstName",
    "ListAgentLastName",
    "ListAgentFullName",
    "BuyerAgentMlsId",
    "BuyerAgentFirstName",
    "BuyerAgentLastName",
    "ListOfficeName",
    "BuyerOfficeName",
    "BuyerOfficeAOR",

    # Identifiers
    "ListingKey",
    "ListingKeyNumeric",
    "ListingId",
    "StreetNumberNumeric",

    # Raw address
    "UnparsedAddress"
]

# Drop only columns that exist (safe practice)
train_df = train_df.drop(columns=[col for col in manual_drop_cols if col in train_df.columns])
test_df  = test_df.drop(columns=[col for col in manual_drop_cols if col in test_df.columns])

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (67746, 29)
Test shape: (10331, 29)


In [9]:
print(train_df.columns)

Index(['ViewYN', 'PoolPrivateYN', 'CloseDate', 'ClosePrice', 'Latitude',
       'Longitude', 'PropertyType', 'LivingArea', 'MLSAreaMajor',
       'CountyOrParish', 'AttachedGarageYN', 'ParkingTotal', 'PropertySubType',
       'LotSizeAcres', 'YearBuilt', 'BathroomsTotalInteger', 'City',
       'BedroomsTotal', 'StateOrProvince', 'FireplaceYN', 'Stories', 'Levels',
       'LotSizeArea', 'NewConstructionYN', 'GarageSpaces',
       'HighSchoolDistrict', 'PostalCode', 'AssociationFee',
       'LotSizeSquareFeet'],
      dtype='object')


### Statistical Feature Selection (Train-driven)

After manual dropping, we apply statistical filters to remove uninformative or redundant features.
**All decisions are computed on `train_df` only**, then applied to both `train_df` and `test_df`
to avoid any leakage from the test set.

---

1. Drop Near-Zero Variance Features

Some features barely change across the dataset (e.g., 99% of values are the same).
These features contribute almost no predictive power and can add noise.

- **Numeric:** drop columns with `nunique <= 1` (constant)
- **Categorical:** drop columns where the most common value accounts for `> 95%` of rows

This improves generalization and prevents the model from wasting capacity on non-informative signals.

---

2. Drop Highly Correlated (Redundant) Numeric Features

Many numeric features may encode the same underlying quantity (e.g., multiple lot size fields).
If two numeric features have `|corr| > 0.90`, we drop one to reduce:

- multicollinearity
- instability in linear models
- redundant information

We keep the feature that is **more correlated with the target** (`ClosePrice`) and drop the other.

---

3. Rank Features by Correlation With the Target (Numeric Only)

We compute correlation between each numeric feature and `ClosePrice` to understand which variables
have stronger linear association with the target.

Notes:
- Correlation only captures **linear** effects.
- We **do not automatically drop** features only because correlation is small (some may be non-linear).
- This step is mainly for **inspection and justification** of feature usefulness.

In [10]:
TARGET = "ClosePrice"

# -------- Near-zero variance --------

low_var_cols = []

for col in train_df.columns:
    if col == TARGET:
        continue
    
    if train_df[col].dtype == "object":
        if train_df[col].value_counts(normalize=True, dropna=False).max() > 0.95:
            low_var_cols.append(col)
    else:
        if train_df[col].nunique(dropna=True) <= 1:
            low_var_cols.append(col)

train_df = train_df.drop(columns=low_var_cols)
test_df  = test_df.drop(columns=[c for c in low_var_cols if c in test_df.columns])

print("Dropped (near-zero variance):", low_var_cols)
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Dropped (near-zero variance): ['PropertyType', 'PropertySubType', 'StateOrProvince']
Train shape: (67746, 26)
Test shape: (10331, 26)


In [11]:
# -------- High correlation (>0.9) --------

numeric_cols = train_df.select_dtypes(include=np.number).columns.tolist()
numeric_cols.remove(TARGET)

corr_matrix = train_df[numeric_cols].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

high_corr_cols = [col for col in upper.columns if any(upper[col] > 0.90)]

train_df = train_df.drop(columns=high_corr_cols)
test_df  = test_df.drop(columns=[c for c in high_corr_cols if c in test_df.columns])

print("Dropped (high correlation):", high_corr_cols)
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Dropped (high correlation): []
Train shape: (67746, 26)
Test shape: (10331, 26)


In [12]:
# --------  Correlation with target --------

numeric_cols = train_df.select_dtypes(include=np.number).columns.tolist()
numeric_cols.remove(TARGET)

target_corr = train_df[numeric_cols].corrwith(train_df[TARGET]).sort_values(
    key=lambda x: x.abs(), ascending=False
)

print(target_corr.head(20))

LivingArea               0.602949
BathroomsTotalInteger    0.556600
BedroomsTotal            0.347876
Stories                  0.192888
AssociationFee           0.163089
Longitude               -0.079982
GarageSpaces             0.046641
LotSizeArea              0.032899
ParkingTotal             0.024660
YearBuilt               -0.016948
LotSizeAcres             0.008017
LotSizeSquareFeet        0.004156
Latitude                -0.001620
dtype: float64


In [13]:
print(train_df.columns)
train_df

Index(['ViewYN', 'PoolPrivateYN', 'CloseDate', 'ClosePrice', 'Latitude',
       'Longitude', 'LivingArea', 'MLSAreaMajor', 'CountyOrParish',
       'AttachedGarageYN', 'ParkingTotal', 'LotSizeAcres', 'YearBuilt',
       'BathroomsTotalInteger', 'City', 'BedroomsTotal', 'FireplaceYN',
       'Stories', 'Levels', 'LotSizeArea', 'NewConstructionYN', 'GarageSpaces',
       'HighSchoolDistrict', 'PostalCode', 'AssociationFee',
       'LotSizeSquareFeet'],
      dtype='object')


,ViewYN,PoolPrivateYN,CloseDate,ClosePrice,Latitude,Longitude,LivingArea,MLSAreaMajor,CountyOrParish,AttachedGarageYN,...,FireplaceYN,Stories,Levels,LotSizeArea,NewConstructionYN,GarageSpaces,HighSchoolDistrict,PostalCode,AssociationFee,LotSizeSquareFeet
3,True,False,2025-06-13,890000.0,34.264692,-117.221040,3000.0,287A - Arrowhead Woods,San Bernardino,True,...,True,2.0,Two,9600.00,True,2.0,Rim of the World,92352,0.0,9600.0
10,True,False,2025-06-30,1876384.0,34.107983,-118.390320,1800.0,C03 - Sunset Strip - Hollywood Hills West,Los Angeles,False,...,True,2.0,Two,10400.00,False,2.0,San Ramon Valley,90046,0.0,10400.0
11,False,False,2025-06-30,4820000.0,37.567434,-122.388226,4270.0,699 - Not Defined,San Mateo,True,...,True,1.0,Two,22505.00,False,3.0,Other,94010,0.0,22505.0
13,True,False,2025-06-30,865000.0,33.906058,-117.777782,1442.0,85 - Yorba Linda,Orange,True,...,True,1.0,One,4800.00,False,2.0,Placentia-Yorba Linda Unified,92886,0.0,4800.0
14,False,False,2025-06-30,875000.0,37.705919,-122.059421,1086.0,699 - Not Defined,Alameda,True,...,True,1.0,One,5500.00,False,4.0,Placentia-Yorba Linda Unified,94546,138.0,5500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134234,True,False,2025-11-26,675000.0,39.450744,-123.760617,1980.0,FBN - Fort Bragg North,Mendocino,False,...,True,2.0,Two,1.83,False,2.0,Lucia Mar Unified,95437,0.0,79714.8
134237,False,False,2025-11-15,805000.0,34.080766,-118.203070,1441.0,677 - Lincoln Hts,Los Angeles,False,...,False,1.0,One,7219.00,False,2.0,Los Angeles Unified,90031,0.0,7219.0
134248,True,False,2025-11-21,700000.0,32.703615,-117.073444,2160.0,92114 - Encanto,San Diego,False,...,True,1.0,One,6900.00,False,2.0,San Diego Unified,92114,0.0,6900.0
134254,True,False,2025-11-05,215000.0,34.501814,-117.775047,1800.0,LLO - Llano,Los Angeles,True,...,False,1.0,One,218781.00,False,0.0,Antelope Valley Union,93544,0.0,218781.0


### Additional Feature Refinement

1. CountyOrParish

Reason for removal:
- Less informative than:
  - Latitude / Longitude (continuous geographic signal)
  - PostalCode (neighborhood-level)



---

2. Levels

Reason for removal:

- Conceptually overlaps with Stories.
- Describes vertical layout style (e.g., "One", "Two", "Multi/Split").
- LivingArea, BedroomsTotal, and BathroomsTotalInteger
  already capture most structural size information.




In [14]:
cols_to_drop = ["CountyOrParish","Level"]

train_df = train_df.drop(columns=[c for c in cols_to_drop if c in train_df.columns])
test_df  = test_df.drop(columns=[c for c in cols_to_drop if c in test_df.columns])

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (67746, 25)
Test shape: (10331, 25)


In [15]:
# California approximate bounds
LAT_MIN, LAT_MAX = 32, 42
LON_MIN, LON_MAX = -124, -114

# Filter train
train_df = train_df[
    (train_df["Latitude"] >= LAT_MIN) &
    (train_df["Latitude"] <= LAT_MAX) &
    (train_df["Longitude"] >= LON_MIN) &
    (train_df["Longitude"] <= LON_MAX)
]

# Filter test
test_df = test_df[
    (test_df["Latitude"] >= LAT_MIN) &
    (test_df["Latitude"] <= LAT_MAX) &
    (test_df["Longitude"] >= LON_MIN) &
    (test_df["Longitude"] <= LON_MAX)
]

print("Train shape after CA filter:", train_df.shape)
print("Test shape after CA filter:", test_df.shape)

Train shape after CA filter: (67727, 25)
Test shape after CA filter: (10324, 25)


In [16]:
# Save cleaned datasets
train_df.to_csv("../processed_data/train_cleaned.csv", index=False)
test_df.to_csv("../processed_data/test_cleaned.csv", index=False)

print("Files successfully saved in 'processed_data' folder.")

Files successfully saved in 'processed_data' folder.


#END: LEXI CHEN